<a href="https://colab.research.google.com/github/nedokormysh/GB_NLP_intro/blob/lesson9/NLP_intro_hw_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Тема “Генерация текстов (языковое моделирование)”

Разобраться с моделькой генерации текста, собрать самим или взять датасет с вебинара и обучить генератор текстов


In [1]:
# Импорт библиотек

import tensorflow as tf

import numpy as np
import os
import time

In [5]:
# Загрузка текста

file = "evgenyi_onegin.txt"

text = open(file, 'rb').read().decode(encoding='utf-8')

print('Длина текста: {} символов'.format(len(text)))

Длина текста: 286984 символов


In [6]:
print(text[:800])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высоких дум и простоты;
                        Но так и быть - рукой пристрастной
                        Прими собранье пестрых глав,
                        Полусмешных, полупечальных,
                        Простонародных, идеальных,
                        Небрежный плод моих забав,
              


In [8]:
# Количество уникальных символов, которые будут составлять наш словарь

vocab = sorted(set(text))
print(f'{len(vocab)} уникальных символов в тексте.')

131 уникальных символов в тексте.


In [9]:
# Создаем сопоставление между символами и их индексами 
# Преобразовываем текст из символьного в индексный вид

char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

len(text_as_int), len(text)

(286984, 286984)

In [10]:
text_as_int[:100]

array([ 71, 110, 104, 109, 116,  99, 112, 103, 115,   1,  87, 104, 115,
       102, 104, 104, 101, 107, 122,   1,  85, 118, 123, 109, 107, 112,
         0,   0,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,  76, 101, 102, 104, 112,
       107, 108,   1,  84, 112, 104, 102, 107, 112,   0,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1])

In [11]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length + 1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

In [12]:
# Сформируем датасет, выделяя предсказываемый текст

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [13]:
# Перемешаем датасет и разделим его на батчи
BATCH_SIZE = 64

EPOCHS = 20

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [14]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [15]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units // 2,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units * 2,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [16]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           33536     
                                                                 
 lstm (LSTM)                 (64, None, 512)           1574912   
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          6295552   
                                                                 
 lstm_2 (LSTM)               (64, None, 2048)          25174016  
                                                                 
 lstm_3 (LSTM)               (64, None, 1024)          12587008  
                                                                 
 dense (Dense)               (64, None, 131)           134275    
                                                                 
Total params: 45,799,299
Trainable params: 45,799,299
No

In [17]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
    
model.compile(optimizer='adam', loss=loss)

In [18]:
!rm -rf ./training_checkpoints
!mkdir ./training_checkpoints
!ls ./training_checkpoints

In [19]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*5,
    save_weights_only=True)

In [20]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
44/44 [==============================] - 40s 593ms/step - loss: 2.3526
Epoch 2/20
44/44 [==============================] - 27s 594ms/step - loss: 1.7820
Epoch 3/20
44/44 [==============================] - 25s 563ms/step - loss: 1.6052
Epoch 4/20
44/44 [==============================] - 25s 563ms/step - loss: 1.4888
Epoch 5/20
44/44 [==============================] - 26s 578ms/step - loss: 1.4296
Epoch 6/20
44/44 [==============================] - 26s 588ms/step - loss: 1.3842
Epoch 7/20
44/44 [==============================] - 26s 577ms/step - loss: 1.3467
Epoch 8/20
44/44 [==============================] - 25s 569ms/step - loss: 1.3231
Epoch 9/20
44/44 [==============================] - 27s 600ms/step - loss: 1.3263
Epoch 10/20
44/44 [==============================] - 28s 630ms/step - loss: 1.2923
Epoch 11/20
44/44 [==============================] - 27s 591ms/step - loss: 1.2629
Epoch 12/20
44/44 [==============================] - 26s 574ms/step - loss: 1.2244
Epoch 13/20
4

In [21]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_20'

In [22]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            33536     
                                                                 
 lstm_4 (LSTM)               (1, None, 512)            1574912   
                                                                 
 lstm_5 (LSTM)               (1, None, 1024)           6295552   
                                                                 
 lstm_6 (LSTM)               (1, None, 2048)           25174016  
                                                                 
 lstm_7 (LSTM)               (1, None, 1024)           12587008  
                                                                 
 dense_1 (Dense)             (1, None, 131)            134275    
                                                                 
Total params: 45,799,299
Trainable params: 45,799,299


Предсказание текста.

In [24]:
def generate_text(model, start_string, temperature=1):
    # Количество генерируемых символов
    num_generate = 700

    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    temperature = 0.8

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [25]:
start_string = "Так думал молодой повеса,"

In [26]:
text_ = generate_text(model, start_string=start_string)
print('temperature = 1')
print(text_)

temperature = 1
Так думал молодой повеса,OщГщ9::ФъiщICЬэ4Ф{ОПN3Э:ШШ4Wэ)0r?ПWХ?Пъэ)4Бэ!)В64?W3HэNRэ:3ЖddэHЭWээW?Q3З0А?Р-э)Э0EDWЭ3эHГэ?Э:8BКщ!Вo24ИRICЧDэFW?э4бYыФwЦщ0щBfDш05lФ?И?D9GБ:0ЖщОщ9?шъщ:W44эыФQFWъCFyFУyQOА4O97Ф?0CC?шщъщFъъhД0ъГэщ??iПCшэщГQfw4BfWWWWшшъчk}Хщ8ШWB1B?Гщ4эщ7У4щ?FWDэщDpэ?3QДщ!)7Лщ0ъ:WэвC?:W6CГW'kщfщQЦ-4ШCCШР16ЮщРFЭFЦъщщI:V?7MхРЭБlAщожит
                        Когда всех горо видает
                        (       И модной сердца подратался
                        Как тем ки она нимогужится. Угужал никой, все пора
                        Возок ее честлою сель                                         Удирить. Ленского вистатьу
                                                                             XX

           


In [27]:
text_ = generate_text(model, start_string=start_string, temperature=2)
print('temperature = 2')
print(text_)

temperature = 2
Так думал молодой повеса,NcэщъRQэ4)RэWэCCRЧЖы{Qэ4?Рэш5VCШщщRqWЭЦъ0FuВ?N-FШ?MQБ-4щЬЮyQDшщ40G?БbДРщщэщщЯщk4CДДlGэшЛ?Ш5k9W!4э?ЖъФq34Ю3RээЬ?fFЦъщ0Ф?cэ?ЦъщщФЮШH49эоэYzWkШш(щщщcp4fДГыDFIРщРэ4шщЮ?э?0nЮщ?Ч10{эRWщ?QyCГWФBTРЦЦCэщщщщnpВ?37B?э4DRшh0D3?TyoР0щщщW?0O7WДщQWQmБ2?D3Q44F;э?Шщэшш9щщщщW4-FWФ?DDFW4TШ?qРrФWГГ7щ9?Жщk43щ?0ЛWwq0ЯCF34WfА3юQ2yЖCШщ!ЭFПHyyэчшД5q9ш?эРыHOuЦWШщiщGWW4499ФшwWъQ7pш:RG{TfЗщщщщ?0ЦЭшФ5ЦRБ9П4EC?uъpЭ4?э?э?eы4FR40iFMъэF2щлiЬFВdDn4DFэ4ГWВБщCЦ0FuэЛЦcэH40шШWы0Fэ?ksШъЗЭщъшщъэ:zfY7AДщC4АHFКfQiэээBN4щ:YrWЬfkDfШщыщWЭ8П?W4ПЭЦФщQnЛщWщW4оэыэщшщhщ24ШФ'Ф4I9?Э3DF2э94э?fээРщощя?yГБщ7B9mWЖ?6эщ?QКрQ4ЦC{ВэkшФ?Ч)7щщэЗcЦCPыйъ9ОGЛэWы?щ7?Cэ?А43")WHicfF0Eэ3vyЖГэiэБ4YщШ:?Ж0ЛэщCЗX(C3?fВ?3ш?QyЖBЛYГRэQщй2щРВщ7Р-4УР?3ЯWщ?ЭB?WWРщщ9?CШ


In [28]:
text_ = generate_text(model, start_string=start_string, temperature=0.1)
print('temperature = 0.1')
print(text_)

temperature = 0.1
Так думал молодой повеса,ъc!эQэ?ВБэПщЕшЯщЛ??{Cs424Dэ8BЦСDkY4Mtesuete eeedete ielile аt          Не верил: глибость небраньем,
                                                XXII

                        На окращьцами замонит.
                        Млегой моей за еслобном мен,
                                                              XIII

                                                           Я не видел нолься не зремя,
                        Да своей гости; где моиен
                        Унычала ворожили был
                        И делького сердце гостим
                        Ни поскной жерое тутой
                        Тловомит новый каким сны,
                                                 


Лучше всего получился текст при 0.1